### Insurance
#### Project Goal: provide a method for predicting the claim cost for each policy

In [5]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

insurance = pd.read_csv("InsNova_data_2023_train.csv")

insurance.head()

,id,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,...,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
0,1,0.77,0.444504,SEDAN,4,M,D,3,petrol,147,...,S,1,weekday,6pm - 12am,6,640.448137,1.0,0,0,0.0
1,2,4.45,0.562183,STNWG,1,M,A,3,petrol,158,...,S,1,weekday,6am - 12pm,12,683.749691,0.0,0,0,0.0
2,3,4.90,0.465244,STNWG,1,F,A,3,petrol,159,...,M,1,weekday,6pm - 12am,6,653.656117,1.0,0,0,0.0
3,4,0.48,0.271039,PANVN,4,M,A,4,petrol,80,...,S,1,weekday,12pm - 6pm,12,642.574671,0.0,0,0,0.0
4,5,0.85,0.141624,SEDAN,4,F,A,5,petrol,126,...,S,0,weekday,6am - 12pm,6,647.175035,0.0,0,0,0.0


In [6]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22619 entries, 0 to 22618
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     22619 non-null  int64  
 1   veh_value              22619 non-null  float64
 2   exposure               22619 non-null  float64
 3   veh_body               22619 non-null  object 
 4   veh_age                22619 non-null  int64  
 5   gender                 22619 non-null  object 
 6   area                   22619 non-null  object 
 7   agecat                 22619 non-null  int64  
 8   engine_type            22619 non-null  object 
 9   max_power              22619 non-null  int64  
 10  driving_history_score  22619 non-null  float64
 11  veh_color              22619 non-null  object 
 12  marital_status         22619 non-null  object 
 13  e_bill                 22619 non-null  int64  
 14  time_of_week_driven    22619 non-null  object 
 15  ti

### EDA

### Transformation

In [9]:
insurance = insurance.drop(["clm", "numclaims"], axis=1)

In [10]:
categorical = insurance.select_dtypes(include=[object])
categorical.head()

,veh_body,gender,area,engine_type,veh_color,marital_status,time_of_week_driven,time_driven
0,SEDAN,M,D,petrol,black,S,weekday,6pm - 12am
1,STNWG,M,A,petrol,white,S,weekday,6am - 12pm
2,STNWG,F,A,petrol,black,M,weekday,6pm - 12am
3,PANVN,M,A,petrol,white,S,weekday,12pm - 6pm
4,SEDAN,F,A,petrol,white,S,weekday,6am - 12pm


In [11]:
# import preprocessing from sklearn
from sklearn import preprocessing

In [12]:
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
categorical_2 = categorical.apply(le.fit_transform)
categorical_2.head()

,veh_body,gender,area,engine_type,veh_color,marital_status,time_of_week_driven,time_driven
0,9,1,3,3,0,1,0,3
1,10,1,0,3,7,1,0,2
2,10,0,0,3,0,0,0,3
3,7,1,0,3,7,1,0,1
4,9,0,0,3,7,1,0,2


In [13]:
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(categorical_2)

# 3. Transform
onehotlabels = enc.transform(categorical_2).toarray()
onehotlabels.shape

(22619, 42)

In [14]:
onehotlabels

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [23]:
insurance.isna().sum().sum()

0

In [24]:
X = insurance.drop("claimcst0", axis=1)
X.head()

,id,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,driving_history_score,veh_color,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind
0,1.0,0.77,0.444504,SEDAN,4.0,M,D,3.0,petrol,147.0,67.0,black,S,1.0,weekday,6pm - 12am,6.0,640.448137,1.0
1,2.0,4.45,0.562183,STNWG,1.0,M,A,3.0,petrol,158.0,76.0,white,S,1.0,weekday,6am - 12pm,12.0,683.749691,0.0
2,3.0,4.90,0.465244,STNWG,1.0,F,A,3.0,petrol,159.0,58.0,black,M,1.0,weekday,6pm - 12am,6.0,653.656117,1.0
3,4.0,0.48,0.271039,PANVN,4.0,M,A,4.0,petrol,80.0,72.0,white,S,1.0,weekday,12pm - 6pm,12.0,642.574671,0.0
4,5.0,0.85,0.141624,SEDAN,4.0,F,A,5.0,petrol,126.0,91.0,white,S,0.0,weekday,6am - 12pm,6.0,647.175035,0.0


In [25]:
y = insurance["claimcst0"]
y.min()

0.0

### Model

In [26]:
from sklearn import tree

clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)

ValueError: could not convert string to float: 'SEDAN'